# Import des librairies nécessaires

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split


# Sommaire

1. partition du dataset (train, test) --> Done
2. best feature
2. kppv

## Partition

In [2]:
data = pd.read_csv("fetal_health.csv")

data.fetal_health = pd.Categorical(data.fetal_health, categories=[1.0, 2.0, 3.0], ordered=True)

In [3]:
# Création de deux DF (un de labels et un de données)
labels = data.fetal_health
clean_data = data.drop('fetal_health', axis=1)


In [4]:
print(clean_data.shape)
print(labels.shape)

(2126, 21)
(2126,)


Les dimensions sont cohérentes

In [5]:
data_train, data_test, labels_train, labels_test = train_test_split(clean_data, labels, test_size=0.2, random_state=123, shuffle=True, stratify=labels)


Création d'une partition des données avec une proportion 80/20, random state spécifié pour avoir la même partition à chauqe appel, stratify= labels pour conserver les proportions de nos labels 

In [6]:
labels_train
labels_test

C:\Users\samma\AppData\Roaming\Python\Python310\site-packages\IPython\lib\pretty.py:778: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  output = repr(obj)


915     1.0
339     3.0
1925    1.0
1277    1.0
1585    1.0
       ... 
1536    1.0
1735    1.0
454     1.0
69      1.0
1557    1.0
Name: fetal_health, Length: 426, dtype: category
Categories (3, float64): [1.0 < 2.0 < 3.0]

## KNeighbors

In [7]:
from sklearn.neighbors import KNeighborsClassifier

In [8]:
clf = KNeighborsClassifier()
clf.fit(data_train, labels_train)
prediction = clf.predict(data_test)

In [9]:
prediction

array([1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 2., 1., 3., 1., 1., 1., 1., 2., 1., 3., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 3., 1., 1., 2., 1., 1.,
       1., 1., 1., 2., 1., 3., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 3., 1., 1., 3., 3., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 3., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 2., 1., 1., 3., 1., 2., 1., 3., 1., 2., 1., 1., 2., 1., 1., 1.,
       1., 1., 1., 2., 1., 2., 1., 2., 1., 1., 1., 2., 1., 1., 1., 3., 1.,
       1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 1.,
       3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 2.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 2., 1.,
       2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1.,
       1., 1., 1., 1., 2.

In [10]:
prediction = pd.Categorical(prediction, categories=[1.0, 2.0, 3.0], ordered=True)

In [20]:
print("Accuracy pour globale")
print(f"Score données d'entrainements: {clf.score(data_train, labels_train)}")
print(f"Score données de test: {clf.score(data_test, labels_test)}")

Accuracy pour globale
Score données d'entrainements: 0.9188235294117647
Score données de test: 0.8849765258215962


In [19]:
print("Accuracy pour les labels 3")
print(f"Score données d'entrainements: {clf.score(data_train[labels_train == 3], labels_train[labels_train == 3])}")
print(f"Score données de test: {clf.score(data_test[labels_test == 3], labels_test[labels_test == 3])}")

Accuracy pour les labels 3
Score données d'entrainements: 0.8156028368794326
Score données de test: 0.7428571428571429


In [21]:
print("Accuracy pour les labels 2")
print(f"Score données d'entrainements: {clf.score(data_train[labels_train == 2], labels_train[labels_train == 2])}")
print(f"Score données de test: {clf.score(data_test[labels_test == 2], labels_test[labels_test == 2])}")


Accuracy pour les labels 2
Score données d'entrainements: 0.6864406779661016
Score données de test: 0.559322033898305


In [22]:
print("Accuracy pour les labels 1")
print(f"Score données d'entrainements: {clf.score(data_train[labels_train == 1], labels_train[labels_train == 1])}")
print(f"Score données de test: {clf.score(data_test[labels_test == 1], labels_test[labels_test == 1])}")

Accuracy pour les labels 1
Score données d'entrainements: 0.9712773998488284
Score données de test: 0.9578313253012049


On réalise donc un K plus proches voisins (k=5) sur l'ensemble d'entrainement et nous utilisons ce modèle (clf) pour prédire les data de tests. En calculant les scores (proportions de bonnes prédictions) nous avons 92% et 89% pour les préditctions de test. La prédisction est donc bonne et se fénéralise bien à de nouvelles données (pas d'overfitting). 

Par contre en regardant le score pour chaque classe, on remarque que celui-ci est très bon pour la classe 1 mais beaucoup moins bon pour la classe 2 et 3

In [12]:
from sklearn.decomposition import PCA
import plotly.express as px

# Réaliser l'analyse en composantes principales (PCA)
cls = PCA(n_components=21)
pcs = cls.fit_transform(data_test)

# Créer la première figure avec un titre
fig_original = px.scatter(x=pcs.T[0], y=pcs.T[1], color=labels_test, 
                          labels={'x': 'PC1', 'y': 'PC2'}, width=650)
fig_original.update_layout(title="Original PCA (2D)")

# Créer la deuxième figure avec un titre
fig_KNeighbors = px.scatter(x=pcs.T[0], y=pcs.T[1], color=prediction, 
                            labels={'x': 'PC1', 'y': 'PC2'}, width=650)
fig_KNeighbors.update_layout(title="KNeighbors PCA (2D)")

# Afficher les figures
fig_original.show()
fig_KNeighbors.show()

